In [1]:
import requests
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta

In [2]:
main = gpd.read_file('./full_globfire/full_globfire.shp')

# Drop bottom 20% area fires
bottom = main['area_ha'].quantile(0.2)
main = main[main['area_ha'] > bottom]

# Add bounding boxes for scrape
main = pd.concat([main, main.bounds], axis=1)

In [ ]:
main.head()

## Vertical Profiles Available


In [4]:
profiles_of_interest = [
    'air',  # Air Temperature
    'hgt',  # Geo-potential Height
    'omega',  # Vertical Velocity in Pressure Coordinates
    'shum',  # Specific Humidity
    'tke',  # Turbulent Kinetic Energy
    'uwnd',  # U-component of wind
    'vwnd'  # V-component of wind
]

In [5]:
import os

for profile in profiles_of_interest:
    os.makedirs(f'./assets/{profile}', exist_ok=True)

In [6]:
def get_relevant_months(event_date):
    year_months = []

    if not isinstance(event_date, datetime):
        event_date = datetime.strptime(
            event_date, "%Y-%m-%d")

    # Add the actual month of the fire
    year_months.append(event_date.strftime("%Y%m"))

    # Check for start of month (if the event is on the 1st or 2nd)
    if event_date.day <= 2:
        previous_month = (event_date.replace(day=1) -
                          timedelta(days=1)).strftime("%Y%m")
        year_months.append(previous_month)

    # Check for end of month (if the event is on the last or second-to-last day)
    next_month = event_date.replace(day=28) + timedelta(days=4)
    last_day = next_month - timedelta(days=next_month.day)
    if event_date.date() >= last_day.date() - timedelta(days=1):
        next_month = (event_date.replace(day=1) +
                      timedelta(days=32)).replace(day=1).strftime("%Y%m")
        year_months.append(next_month)

    return year_months

In [7]:
BASE_URL = "https://psl.noaa.gov/thredds/fileServer/Datasets/NARR/pressure/{}.{}.nc"

In [8]:
def download_file(url, save_path):
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

In [ ]:
for _, row in main.iterrows():
    year_months = get_relevant_months(row['initialdat'])

    for month in year_months:
        for profile in profiles_of_interest:
            url = BASE_URL.format(profile, month)
            save_path = os.path.join(
                'assets', profile, f"{profile}.{month}.nc")

            try:
                download_file(url, save_path)
                print(f"Successfully downloaded {profile} data for {month}")
            except requests.RequestException as e:
                print(f"Error downloading {profile} data for {month}: {e}")
                break